<a href="https://colab.research.google.com/github/pavansai26/web-scrapping/blob/master/celery%2C_flask%2C_mongo_db_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Flask
!pip install celery
!pip install flask-mongoengine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.6/405.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.6/110.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.5/136.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.1/492.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 24.6 MB/s eta 0:00:00


In [2]:
from flask import Flask, request
from celery import Celery
from flask_mongoengine import MongoEngine
import time


In [ ]:
# Initialize Flask app
app = Flask(__name__)

# Initialize Celery app
celery = Celery(app.name, broker='amqp://guest@localhost//', backend='mongodb://localhost:27017/celery')

db = MongoEngine()

# Define Flask route to submit task for processing
@app.route('/process', methods=['POST'])
def process_task():
    # Get task data from request JSON
    task_data = request.json
    # Submit task to Celery for processing using the `process_data` task
    result = process_data.delay(task_data)
    # Return task ID to user
    return {'task_id': result.id}

# Define Celery task to process data asynchronously
@celery.task
def process_data(task_data):
    # Simulate some processing time
    time.sleep(10)
    # Perform processing with the task data
    result_data = task_data * 2
    # Save the result to MongoDB using the `Results` model
    result = Results(data=result_data)
    result.save()
    # Return ID of the result document to the calling function
    return result.id

# Define MongoDB model to store results
class Results(db.Document):
    data = db.StringField(required=True)

# Start the Celery worker process
celery -A app.celery worker --loglevel=info

# Start the Flask application
if __name__ == '__main__':
    app.run(debug=True)
